# Working Code

## Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## Getting Data

In [ ]:
tracking_week_1 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv")

In [ ]:
play1 = tracking_week_1[(tracking_week_1.gameId == 2022091200) & (tracking_week_1.playId == 64)]

before_snap = play1[play1.frameType == "BEFORE_SNAP"]
snap = play1[play1.frameType == "SNAP"]
after_snap = play1[play1.frameType == "AFTER_SNAP"]

## Functions to get the player that has the ball

In [ ]:
def ball_distance_dictionary(player_df : pd.DataFrame) -> (dict, int): 
    player_sep = {}
    players = player_df.displayName.unique()
    football = player_df[player_df.displayName == 'football'].reset_index(drop=True)
    for i in range(len(player_df.displayName.unique())):
        player = player_df[player_df.displayName == players[i]].reset_index(drop=True)
        if player.displayName[0] != "football":
            distances = list(zip(player.x, player.y))
            seps = []
            for idx, (x_dist, y_dist) in enumerate(distances):
                sep = ((football.x[idx] - x_dist)**2 + (football.y[idx] - y_dist)**2)
                seps.append(sep)
                frames = len(seps)
        player_sep[players[i]] = seps
    return player_sep, frames

def player_with_ball(player_df : pd.DataFrame) -> list:
    player_sep, frames = ball_distance_dictionary(player_df)
    player_with_ball = [None] * frames
    players = player_df.displayName.unique()
    
    for frame in range(frames):
        curr = None
        curr_dist = 17209
        for player in players:
            if player_sep[player][frame] < 1:
                curr = player
                curr_dist = player_sep[player][frame]
            if curr is not None and player_sep[player][frame] < curr_dist:
                curr = player
                curr_dist = player_sep[player][frame]
        if curr is not None:
            player_with_ball[frame] = (curr, curr_dist)
        else:
            player_with_ball[frame] = ('In air', None)

    return player_with_ball

## Making graphs of the play that label the player with the ball

In [ ]:
frames = []
i = 0
ball_posessions = player_with_ball(play1)
for frame in play1.frameId.unique():
    #frames.append([plt.figure()])
    play_at_frame = play1[play1.frameId == frame].sort_values(by="jerseyNumber").reset_index(drop=True)
    players = play_at_frame[:-1]
    ball = play_at_frame[-1:] 

    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    plt.scatter(players.x, players.y, c=range(len(players)))
    plt.scatter(ball.x, ball.y, c="red", marker="*") 
    plt.title(play_at_frame.frameType[0])

    player = ball_posessions[i][0]
    if (player != "In air"): 
        player_x = list(play_at_frame[play_at_frame.displayName == player].x)[0]
        player_y = list(play_at_frame[play_at_frame.displayName == player].y)[0]
        plt.text(player_x, player_y, player)
        
    
    #frames.append(plt.gcf())
    plt.savefig(f"img{i}.png")
    plt.close()
    i += 1

In [ ]:
play1.frameId.unique()

### Calculating frame rate

In [ ]:
play_length = (pd.to_datetime(play1.time.max()) - pd.to_datetime(play1.time.min())).total_seconds()
num_frames = len(play1.frameId.unique())
print(f"Frame rate: {num_frames / play_length}")

## Create animation from PNGs

In [ ]:
os.system("ffmpeg -r 10.06 -i img%01d.png -vcodec mpeg4 -y movie.mp4")

### Delete PNGs

In [ ]:
#courtesy of Gemini
def delete_png_files(folder_path):
    """Deletes all PNG files in the specified folder."""

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            file_path = os.path.join(folder_path, filename)
            try:
                os.remove(file_path)
            except OSError as e:
                print(f"Error deleting {file_path}: {e}")

# Specify the folder path where you want to delete PNG files
folder_path = "/kaggle/working" 

# Call the function to delete PNG files
delete_png_files(folder_path)

# Testing / Experiments

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

In [ ]:
games = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/games.csv")
player_play = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/player_play.csv")
players = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/players.csv")
plays = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/plays.csv")
tracking_week_1 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv")
#tracking_week_2 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv")
#tracking_week_3 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv")
#tracking_week_4 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv")
#tracking_week_5 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv")
#tracking_week_6 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv")
#tracking_week_7 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv")
#tracking_week_8 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv")
#tracking_week_9 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv")

In [ ]:
games.head()

In [ ]:
print(player_play.shape)
player_play.head()

In [ ]:
print(players.shape)
players.head()

In [ ]:
print(plays.shape)
plays.head()

In [ ]:
plays.sort_values(by=["gameId", "playId"])

In [ ]:
print(tracking_week_1.shape)
tracking_week_1.head()

In [ ]:
tracking_week_1.gameId.unique()

In [ ]:
tracking_week_1.playDirection.unique()

In [ ]:
tracking_week_1[tracking_week_1.gameId == 2022091200].playId.unique()

In [ ]:
tracking_week_1.frameType.unique()

In [ ]:
play1 = tracking_week_1[(tracking_week_1.gameId == 2022091200) & (tracking_week_1.playId == 64)]

In [ ]:
print(tracking_week_1.x.aggregate(["min", "max"]))
print(tracking_week_1.y.aggregate(["min", "max"]))

In [ ]:
before_snap = play1[play1.frameType == "BEFORE_SNAP"]
snap = play1[play1.frameType == "SNAP"]
after_snap = play1[play1.frameType == "AFTER_SNAP"]

In [ ]:
players = before_snap.displayName.unique()
players

In [ ]:
player0 = before_snap[before_snap.displayName == players[0]]
player0

In [ ]:
player0.loc[:, ["x", "y"]].isna().sum()

In [ ]:
player0.time.aggregate(["min", "max"])

In [ ]:
player0.displayName.reset_index(drop=True)

In [ ]:
n = 0
frames = []
for frameType in [before_snap, snap, after_snap]:
    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    for i in range(len(frameType.displayName.unique())):
        player = frameType[frameType.displayName == players[i]].reset_index(drop=True)
        if player.displayName[0] == "football":
            plt.scatter(player.x, player.y, c="red")
            plt.text(list(player.x)[-1], list(player.y)[-1], "FB")
        else:
            plt.scatter(player.x, player.y, c=player.frameId)
            #plt.text(list(player.x)[-1], list(player.y)[-1], list(player.jerseyNumber)[-1])
    
    title = ""
    match n:
        case 0: 
            title = "before snap"
        case 1: 
            title = "snap"
        case 2: 
            title = "after snap"
    plt.title(title)
    
    plt.show()
    n+=1

In [ ]:
after_snap = play1[play1.frameType == "AFTER_SNAP"]
print(len(player_with_ball(play1)))
player_with_ball(play1)